In [53]:
import tensorflow as tf
from tensorflow import image as img
import os
import shutil, sys
import numpy as np
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
import keras
from keras.utils import plot_model
from keras.applications.mobilenet import MobileNet
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

In [54]:
from keras.models import load_model
model = load_model('keras_aug_4_15_12.h5')

In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              18875392  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_4 (Dense)              (None, 12)                780       
Total para

In [56]:
model.layers[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 196, 196, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 196, 196, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 196, 196, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 98, 98, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 98, 98, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 98, 98, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 49, 49, 128)       0         
__________

In [25]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [61]:
input = Input(shape=(196,196,3))
x = model.get_layer("vgg19")(input)
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.3) (x)
x = Dense(256, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(2, activation='softmax') (x)

model= Model(input, preds)

for layer in model.layers:
    if layer.name == 'vgg19':
        layer.trainable = False
model.compile(optimizer=optimizers.RMSprop(lr=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [62]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 196, 196, 3)       0         
_________________________________________________________________
vgg19 (Model)                (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten_4 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              18875392  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_15 (Dense)             (None, 64)                16448     
__________

In [63]:
model.layers[1].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 196, 196, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 196, 196, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 196, 196, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 98, 98, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 98, 98, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 98, 98, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 49, 49, 128)       0         
__________

In [64]:
image_size = 196
batch_size = 30
train2_dir = './train2'
val2_dir = './validation2'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator_2 = train_datagen.flow_from_directory(
                    train2_dir,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator_2 = validation_datagen.flow_from_directory(
                        val2_dir,
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 857 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [65]:
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2)

Epoch 1/10
 - 24s - loss: 0.5168 - acc: 0.8085 - val_loss: 0.7071 - val_acc: 0.8000
Epoch 2/10
 - 22s - loss: 0.2301 - acc: 0.9038 - val_loss: 0.5279 - val_acc: 0.8167
Epoch 3/10
 - 22s - loss: 0.2410 - acc: 0.9017 - val_loss: 0.5747 - val_acc: 0.8333
Epoch 4/10
 - 22s - loss: 0.2099 - acc: 0.9147 - val_loss: 1.1252 - val_acc: 0.8167
Epoch 5/10
 - 22s - loss: 0.2374 - acc: 0.9057 - val_loss: 0.9210 - val_acc: 0.7833
Epoch 6/10
 - 22s - loss: 0.2002 - acc: 0.9264 - val_loss: 0.6268 - val_acc: 0.8667
Epoch 7/10
 - 22s - loss: 0.1815 - acc: 0.9252 - val_loss: 0.9567 - val_acc: 0.8167
Epoch 8/10
 - 22s - loss: 0.1914 - acc: 0.9252 - val_loss: 0.6066 - val_acc: 0.8333
Epoch 9/10
 - 22s - loss: 0.2041 - acc: 0.9216 - val_loss: 0.5735 - val_acc: 0.8667
Epoch 10/10
 - 22s - loss: 0.1503 - acc: 0.9336 - val_loss: 0.9866 - val_acc: 0.8333


In [66]:
model.save('2clasifier_10_epochs.h5')

In [67]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2)

Epoch 1/10
 - 23s - loss: 0.5178 - acc: 0.8790 - val_loss: 0.6855 - val_acc: 0.7833
Epoch 2/10
 - 22s - loss: 0.2407 - acc: 0.9043 - val_loss: 1.1232 - val_acc: 0.7667
Epoch 3/10
 - 22s - loss: 0.2290 - acc: 0.9206 - val_loss: 0.6427 - val_acc: 0.8167
Epoch 4/10
 - 23s - loss: 0.2490 - acc: 0.9006 - val_loss: 0.7012 - val_acc: 0.8167
Epoch 5/10
 - 22s - loss: 0.2950 - acc: 0.8920 - val_loss: 0.4910 - val_acc: 0.7500
Epoch 6/10
 - 23s - loss: 0.2286 - acc: 0.9048 - val_loss: 0.4771 - val_acc: 0.8500
Epoch 7/10
 - 22s - loss: 0.2427 - acc: 0.9098 - val_loss: 0.6937 - val_acc: 0.7833
Epoch 8/10
 - 22s - loss: 0.2503 - acc: 0.8894 - val_loss: 0.4896 - val_acc: 0.8167
Epoch 9/10
 - 22s - loss: 0.2213 - acc: 0.9086 - val_loss: 0.4751 - val_acc: 0.8333
Epoch 10/10
 - 23s - loss: 0.2417 - acc: 0.9152 - val_loss: 0.4498 - val_acc: 0.8500


In [68]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2)

Epoch 1/10
 - 24s - loss: 0.1834 - acc: 0.9198 - val_loss: 0.5354 - val_acc: 0.8333
Epoch 2/10
 - 21s - loss: 0.1625 - acc: 0.9382 - val_loss: 0.5456 - val_acc: 0.8500
Epoch 3/10
 - 23s - loss: 0.1444 - acc: 0.9509 - val_loss: 0.6095 - val_acc: 0.8333
Epoch 4/10
 - 22s - loss: 0.1374 - acc: 0.9436 - val_loss: 0.6924 - val_acc: 0.8500
Epoch 5/10
 - 22s - loss: 0.1330 - acc: 0.9425 - val_loss: 0.5729 - val_acc: 0.8500
Epoch 6/10
 - 23s - loss: 0.1521 - acc: 0.9367 - val_loss: 0.6035 - val_acc: 0.8500
Epoch 7/10
 - 21s - loss: 0.1451 - acc: 0.9285 - val_loss: 0.6076 - val_acc: 0.8333
Epoch 8/10
 - 23s - loss: 0.1433 - acc: 0.9367 - val_loss: 0.6350 - val_acc: 0.8500
Epoch 9/10
 - 22s - loss: 0.1236 - acc: 0.9436 - val_loss: 0.7460 - val_acc: 0.8333
Epoch 10/10
 - 23s - loss: 0.1150 - acc: 0.9563 - val_loss: 0.6241 - val_acc: 0.8500


In [69]:
model.save('2clasifier_10_epochs.h5')

In [74]:
class_weight = {0: 2.67,
                1: 1.0}

In [78]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.0000005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.1960 - acc: 0.9509 - val_loss: 0.4084 - val_acc: 0.8500
Epoch 2/10
 - 21s - loss: 0.2374 - acc: 0.9482 - val_loss: 0.3395 - val_acc: 0.8500
Epoch 3/10
 - 23s - loss: 0.2188 - acc: 0.9452 - val_loss: 0.3588 - val_acc: 0.8500
Epoch 4/10
 - 23s - loss: 0.1931 - acc: 0.9609 - val_loss: 0.4080 - val_acc: 0.8833
Epoch 5/10
 - 20s - loss: 0.1630 - acc: 0.9540 - val_loss: 0.4406 - val_acc: 0.8833
Epoch 6/10
 - 24s - loss: 0.1872 - acc: 0.9463 - val_loss: 0.4480 - val_acc: 0.8833
Epoch 7/10
 - 22s - loss: 0.1577 - acc: 0.9609 - val_loss: 0.5272 - val_acc: 0.8667
Epoch 8/10
 - 21s - loss: 0.1788 - acc: 0.9555 - val_loss: 0.6430 - val_acc: 0.8500
Epoch 9/10
 - 23s - loss: 0.1986 - acc: 0.9535 - val_loss: 0.5387 - val_acc: 0.8833
Epoch 10/10
 - 22s - loss: 0.1601 - acc: 0.9597 - val_loss: 0.4772 - val_acc: 0.8667


In [79]:
model.save('2clasifier_30_epochs.h5')

In [80]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.0000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 24s - loss: 0.1571 - acc: 0.9503 - val_loss: 0.4847 - val_acc: 0.8833
Epoch 2/10
 - 21s - loss: 0.1935 - acc: 0.9535 - val_loss: 0.4803 - val_acc: 0.8833
Epoch 3/10
 - 23s - loss: 0.1981 - acc: 0.9436 - val_loss: 0.4762 - val_acc: 0.8833
Epoch 4/10
 - 21s - loss: 0.1820 - acc: 0.9586 - val_loss: 0.4723 - val_acc: 0.8833
Epoch 5/10
 - 22s - loss: 0.2414 - acc: 0.9517 - val_loss: 0.4412 - val_acc: 0.8833
Epoch 6/10
 - 22s - loss: 0.1995 - acc: 0.9451 - val_loss: 0.4400 - val_acc: 0.8833
Epoch 7/10
 - 21s - loss: 0.2180 - acc: 0.9486 - val_loss: 0.4173 - val_acc: 0.8833
Epoch 8/10
 - 23s - loss: 0.1838 - acc: 0.9546 - val_loss: 0.4131 - val_acc: 0.8833
Epoch 9/10
 - 23s - loss: 0.1973 - acc: 0.9463 - val_loss: 0.4031 - val_acc: 0.8833
Epoch 10/10
 - 21s - loss: 0.1722 - acc: 0.9515 - val_loss: 0.4135 - val_acc: 0.8833


In [81]:
model.save('2clasifier_40_epochs.h5')

In [82]:
model.layers[1].trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.00000005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 26s - loss: 0.1954 - acc: 0.9505 - val_loss: 0.4366 - val_acc: 0.8667
Epoch 2/10
 - 21s - loss: 0.1312 - acc: 0.9666 - val_loss: 0.4517 - val_acc: 0.8667
Epoch 3/10
 - 22s - loss: 0.1798 - acc: 0.9601 - val_loss: 0.4378 - val_acc: 0.8833
Epoch 4/10
 - 23s - loss: 0.1628 - acc: 0.9551 - val_loss: 0.4432 - val_acc: 0.8833
Epoch 5/10
 - 21s - loss: 0.1332 - acc: 0.9597 - val_loss: 0.4485 - val_acc: 0.8833
Epoch 6/10
 - 23s - loss: 0.1870 - acc: 0.9512 - val_loss: 0.4512 - val_acc: 0.8833
Epoch 7/10
 - 23s - loss: 0.1900 - acc: 0.9572 - val_loss: 0.4435 - val_acc: 0.8833
Epoch 8/10
 - 21s - loss: 0.1869 - acc: 0.9624 - val_loss: 0.4502 - val_acc: 0.8667
Epoch 9/10
 - 22s - loss: 0.2087 - acc: 0.9523 - val_loss: 0.4432 - val_acc: 0.8833
Epoch 10/10
 - 22s - loss: 0.2036 - acc: 0.9552 - val_loss: 0.4327 - val_acc: 0.8833


In [83]:
model.save('2clasifier_50_epochs.h5')

In [71]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        val2_dir,
        target_size=(196, 196),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

Found 60 images belonging to 2 classes.


In [72]:
predict_labels = np.argmax(predict, axis=1)
val_labels = np.zeros(predict_labels.size, dtype=int)
for i in range(2):
    for j in range(30):
        val_labels[i*30+j] = int(i);

In [73]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(val_labels, predict_labels)

array([[22,  8],
       [ 1, 29]])